In [17]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
import pandas as pd
import math
import socket
import time

# TensorFlow

In [20]:
### Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

### Load Model
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

### Load Video
video_path = '../../data/videos/'
sampleVideo = video_path + 'sampleVideo.mp4'
basicFour = video_path + 'basic_four.mp4'
basicTwo = video_path + 'basic_two.mp4'
swingDiff = video_path + 'swing_diff.mp4'
swingSame = video_path + 'swing_same.mp4'
swingRaw = video_path + 'swingRaw.mp4'
temporalDifficult = video_path + 'temporalDifficultRaw.mp4'
temporalEasy = video_path + 'temporalEasyRaw.mp4'



### Draw EDGES
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

### Vector List
vectorList = [
    [0,1],
    [0,2],
    [1,3],
    [2,4],
    [3,5],
    [0,6],
    [1,7],
    [6,7],
    [6,8],
    [7,9],
    [8,10],
    [9,10]
]

# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)


def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 3, (0,255,0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


###
# Variables for drawing plot in real-time


if __name__ == "__main__":
    ### Variables
    numberOfPeople = 6
    lamdaVal = 0.885

    minBPD = 10.0
    maxBPD = 0.0
    
    saveNum = 0
    saveBPD = []

    ### Loading Video File
    cap = cv2.VideoCapture(temporalEasy)    ### Change the File Here!!
    while cap.isOpened():
        ret, frame = cap.read()

        ### Variables for each frame
        BPD = []
        
        # Resize image
        img = frame.copy()
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
        input_img = tf.cast(img, dtype=tf.int32)
        
        # Detection section
        results = movenet(input_img)
        keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
        keypoints_with_scores = keypoints_with_scores[:numberOfPeople]
        keypoints_only = np.delete(keypoints_with_scores,2,2)
        keypoints_only_body = np.delete(keypoints_only, [0,1,2,3,4], 1)
        
        print(keypoints_with_scores)



        # Render keypoints 
        loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
#         loop_through_people(frame, [keypoints_with_scores[0]], EDGES, 0.1)    # Check for first person.....

        
        cv2.imshow('Movenet Multipose', frame)
        
        
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

[[[0.40263897 0.91577095 0.7576814 ]
  [0.3972788  0.91948634 0.5443643 ]
  [0.39894855 0.91255    0.4900534 ]
  [0.4038157  0.9231429  0.64102554]
  [0.4057015  0.90904915 0.7117422 ]
  [0.43096387 0.93410677 0.9037962 ]
  [0.4325912  0.8989521  0.86352086]
  [0.42681193 0.9611966  0.7272117 ]
  [0.43518525 0.8760786  0.59941787]
  [0.4174626  0.9826479  0.7220818 ]
  [0.4309015  0.8540149  0.78343725]
  [0.51426613 0.9259904  0.90616983]
  [0.51202166 0.90637535 0.86833423]
  [0.57910514 0.9316432  0.88065207]
  [0.57562256 0.90221816 0.91484237]
  [0.63291943 0.9348199  0.9006905 ]
  [0.6291499  0.89738935 0.8450321 ]]

 [[0.40333614 0.08336502 0.7138668 ]
  [0.3988187  0.08646297 0.5339356 ]
  [0.3989105  0.08056959 0.59946626]
  [0.40436286 0.09064956 0.6575931 ]
  [0.40543947 0.07703294 0.6785532 ]
  [0.43197733 0.10301357 0.90192586]
  [0.43183252 0.06771537 0.88980913]
  [0.42481244 0.12988901 0.77320665]
  [0.43311775 0.04259681 0.7985779 ]
  [0.4188261  0.14612286 0.6744809 ]

[[[0.40275908 0.9155372  0.75116676]
  [0.39753887 0.9192135  0.52689373]
  [0.3986141  0.91267484 0.48156124]
  [0.4038034  0.92293024 0.65236527]
  [0.40566036 0.9090193  0.7063025 ]
  [0.43050057 0.9340994  0.9038318 ]
  [0.43238637 0.899079   0.86763513]
  [0.4261547  0.9621283  0.74372995]
  [0.43512    0.875519   0.5819787 ]
  [0.41634548 0.983537   0.7156956 ]
  [0.43089494 0.8540697  0.7858503 ]
  [0.5143863  0.925943   0.9052489 ]
  [0.51206726 0.9063439  0.8710982 ]
  [0.57919824 0.9315915  0.8770459 ]
  [0.5758403  0.9021559  0.9156878 ]
  [0.6329446  0.9348794  0.9001914 ]
  [0.62933016 0.8974004  0.8462432 ]]

 [[0.4036269  0.0830039  0.6900504 ]
  [0.39899904 0.08608758 0.5955662 ]
  [0.39921057 0.08023574 0.6372708 ]
  [0.40436003 0.09033584 0.667838  ]
  [0.40550023 0.07697828 0.6784247 ]
  [0.43193036 0.10288262 0.90197015]
  [0.43187183 0.06760789 0.8889959 ]
  [0.4249866  0.12958835 0.7814803 ]
  [0.43340605 0.04234202 0.80238086]
  [0.41882473 0.14570493 0.683698  ]

[[[0.40261084 0.91562694 0.7640685 ]
  [0.39719647 0.9193558  0.5399123 ]
  [0.39892066 0.9124242  0.47806215]
  [0.40374288 0.9229623  0.6595191 ]
  [0.40564442 0.9089312  0.7167973 ]
  [0.4303101  0.93402445 0.90494245]
  [0.4323876  0.89878017 0.8819669 ]
  [0.42585653 0.9623968  0.7499757 ]
  [0.43516248 0.87491673 0.57742244]
  [0.41616186 0.9842206  0.7087891 ]
  [0.43087447 0.8539354  0.7977459 ]
  [0.5144422  0.9260066  0.90688413]
  [0.5122975  0.90625614 0.87704647]
  [0.57919145 0.9316107  0.8777403 ]
  [0.576033   0.9020905  0.9146363 ]
  [0.63289005 0.9348655  0.90286344]
  [0.6293602  0.8973773  0.8494323 ]]

 [[0.40169364 0.2504575  0.5944823 ]
  [0.39814734 0.25360402 0.5833349 ]
  [0.39734596 0.24775703 0.6135362 ]
  [0.40497538 0.25698265 0.61403495]
  [0.40571377 0.24449824 0.6101708 ]
  [0.4293787  0.26796946 0.805589  ]
  [0.43231362 0.23385751 0.8972352 ]
  [0.4274974  0.29212233 0.68601894]
  [0.43601367 0.21307452 0.5785861 ]
  [0.42138147 0.31348464 0.69577515]

[[[0.40295053 0.91569644 0.7560113 ]
  [0.3973862  0.9193821  0.54920024]
  [0.39926684 0.91245    0.48274547]
  [0.40377682 0.9228838  0.6586918 ]
  [0.40591678 0.9088421  0.70205146]
  [0.43031764 0.93381876 0.90169936]
  [0.43258554 0.89901894 0.872476  ]
  [0.426131   0.96171486 0.7669876 ]
  [0.43538928 0.8754493  0.5813401 ]
  [0.41629896 0.9842456  0.7050118 ]
  [0.43129814 0.85358685 0.8039393 ]
  [0.51472473 0.9260166  0.90838504]
  [0.51255697 0.9063148  0.87458843]
  [0.57940894 0.9315726  0.8730393 ]
  [0.5758954  0.90211815 0.91385186]
  [0.63291633 0.9349887  0.9034167 ]
  [0.6293379  0.89739066 0.84657663]]

 [[0.40345642 0.08279454 0.68636817]
  [0.39875993 0.08588648 0.5913436 ]
  [0.3989511  0.08008044 0.63254464]
  [0.40398562 0.09019723 0.6966422 ]
  [0.40503034 0.07681169 0.70052415]
  [0.43092775 0.1027495  0.910649  ]
  [0.43044612 0.06740744 0.89837164]
  [0.4237793  0.13056728 0.7676559 ]
  [0.43284804 0.04180768 0.82982826]
  [0.41860718 0.14761154 0.64334613]

[[[0.40299696 0.9157092  0.74516004]
  [0.39756247 0.9193079  0.5464527 ]
  [0.3993139  0.91246986 0.49231395]
  [0.40400124 0.9228075  0.649357  ]
  [0.40604424 0.908912   0.69940114]
  [0.4306131  0.9337374  0.8994439 ]
  [0.43268082 0.8990728  0.8697495 ]
  [0.42649877 0.96129936 0.75186896]
  [0.4355659  0.87537825 0.59212863]
  [0.41692096 0.9829177  0.7007734 ]
  [0.4312824  0.8535515  0.80671245]
  [0.51468045 0.9260147  0.90776575]
  [0.512519   0.9062912  0.87395084]
  [0.5792844  0.9315843  0.8759798 ]
  [0.5756235  0.9022569  0.9124156 ]
  [0.6327731  0.9349618  0.90202117]
  [0.6292419  0.89737827 0.8342905 ]]

 [[0.4036417  0.08283842 0.6852812 ]
  [0.39897415 0.08593296 0.60063463]
  [0.39917165 0.08011942 0.6350602 ]
  [0.40408143 0.09018289 0.6956098 ]
  [0.40515345 0.07678956 0.693586  ]
  [0.43104273 0.10256033 0.9096037 ]
  [0.43065822 0.06734338 0.89731264]
  [0.42339727 0.1302798  0.7792767 ]
  [0.43298483 0.04200557 0.824925  ]
  [0.4186747  0.14636987 0.65032154]

...

[[[0.40270436 0.4151462  0.759771  ]
  [0.39796826 0.41889665 0.48413298]
  [0.3983427  0.41253635 0.5081702 ]
  [0.40491205 0.42219815 0.646607  ]
  [0.40561494 0.4090545  0.7021047 ]
  [0.43135655 0.4319543  0.891925  ]
  [0.4331633  0.399292   0.8814364 ]
  [0.43188286 0.4577941  0.7858505 ]
  [0.43517137 0.3746712  0.5289113 ]
  [0.43227744 0.48590446 0.73098165]
  [0.43140352 0.3533785  0.7901905 ]
  [0.5151813  0.42520815 0.89092255]
  [0.5129285  0.40624505 0.8611604 ]
  [0.580083   0.43084747 0.77089876]
  [0.5763457  0.40129372 0.9140298 ]
  [0.6354518  0.43533784 0.8717248 ]
  [0.62945753 0.39778626 0.82350934]]

 [[0.40466094 0.91555065 0.66501904]
  [0.3999583  0.9189296  0.56428653]
  [0.40040976 0.9125538  0.51835555]
  [0.40639386 0.92228043 0.5602659 ]
  [0.40717548 0.9086482  0.6482664 ]
  [0.43124017 0.93275017 0.89131427]
  [0.43346983 0.89890665 0.87359047]
  [0.42855775 0.95928854 0.755521  ]
  [0.43506503 0.87607014 0.54158926]
  [0.42077184 0.9777374  0.78157383]

[[[0.4045323  0.91553557 0.6736419 ]
  [0.3998272  0.9189376  0.5636811 ]
  [0.4003025  0.91256255 0.5089349 ]
  [0.40637836 0.9225969  0.55869323]
  [0.4071554  0.9085957  0.65160143]
  [0.43118024 0.93290484 0.889901  ]
  [0.43388197 0.8988206  0.87171984]
  [0.42906618 0.9593919  0.760561  ]
  [0.43560347 0.8761172  0.53432584]
  [0.42064792 0.9786483  0.7769273 ]
  [0.43430263 0.8537944  0.7541685 ]
  [0.5133945  0.9259929  0.8984718 ]
  [0.5119039  0.90624446 0.8673727 ]
  [0.5786054  0.9312603  0.8483669 ]
  [0.57463616 0.90274525 0.8848273 ]
  [0.6321641  0.9348815  0.90087056]
  [0.628577   0.8975344  0.8140383 ]]

 [[0.4028078  0.41499576 0.75857717]
  [0.39808583 0.4187828  0.47136807]
  [0.39843607 0.41244528 0.5275115 ]
  [0.40498877 0.422142   0.64866817]
  [0.4057116  0.40901405 0.70052433]
  [0.43138492 0.4319651  0.8896112 ]
  [0.43319833 0.39918643 0.8896327 ]
  [0.43197474 0.45773482 0.7848087 ]
  [0.43473428 0.37622476 0.5675022 ]
  [0.43229836 0.48583937 0.73301727]

[[[0.40419465 0.91549844 0.6924628 ]
  [0.39946413 0.9189976  0.5644586 ]
  [0.3998015  0.91261226 0.48562503]
  [0.40609586 0.92245877 0.5718469 ]
  [0.40706557 0.90872157 0.6303183 ]
  [0.43055212 0.93310827 0.89111584]
  [0.43317652 0.89877284 0.8781288 ]
  [0.42754194 0.9601426  0.74421513]
  [0.43522584 0.87697166 0.5679386 ]
  [0.41850793 0.9810013  0.7391282 ]
  [0.43315017 0.8541996  0.75615436]
  [0.5132539  0.9259281  0.89779377]
  [0.51181054 0.9061349  0.86876523]
  [0.578485   0.9314036  0.8495397 ]
  [0.5745932  0.90287495 0.8900886 ]
  [0.6320872  0.93482095 0.9006139 ]
  [0.6287317  0.89743245 0.8165569 ]]

 [[0.4025455  0.4155042  0.75916165]
  [0.39785108 0.4191331  0.50882727]
  [0.3982428  0.41281015 0.46932676]
  [0.40475917 0.42217684 0.6496942 ]
  [0.40561384 0.40907255 0.70299715]
  [0.43149135 0.43188754 0.8901221 ]
  [0.43321127 0.39945546 0.8782377 ]
  [0.4319358  0.4576419  0.77829146]
  [0.4349004  0.37622023 0.5652733 ]
  [0.43225425 0.48504874 0.72986436]

[[[0.402633   0.41552517 0.752793  ]
  [0.39798722 0.41912937 0.51067   ]
  [0.39832562 0.41286492 0.46705347]
  [0.404828   0.42213115 0.6489136 ]
  [0.40561816 0.4091414  0.70309526]
  [0.4315849  0.43197584 0.8910206 ]
  [0.43318135 0.39953184 0.8768994 ]
  [0.43230724 0.4578955  0.7850832 ]
  [0.43492934 0.37601224 0.57939726]
  [0.43244296 0.4860572  0.72847646]
  [0.43162268 0.35320136 0.7805451 ]
  [0.51474553 0.4254302  0.8955065 ]
  [0.5123272  0.40630937 0.8586108 ]
  [0.5798237  0.43082175 0.7779831 ]
  [0.5753932  0.40147972 0.9149149 ]
  [0.63507706 0.43528333 0.8781778 ]
  [0.6298204  0.39792272 0.82518464]]

 [[0.40397948 0.9156453  0.6998627 ]
  [0.39923495 0.91903514 0.57647574]
  [0.4001132  0.9128503  0.45436314]
  [0.40490526 0.92265797 0.5929472 ]
  [0.40569255 0.9098878  0.6413294 ]
  [0.42776895 0.93249863 0.8400643 ]
  [0.42794585 0.90032214 0.7844348 ]
  [0.40844554 0.9511992  0.6861217 ]
  [0.41132402 0.88454324 0.66748255]
  [0.38612506 0.96798354 0.7464528 ]

[[[0.4026161  0.41565272 0.7646691 ]
  [0.39780658 0.4192898  0.53103244]
  [0.3976701  0.412741   0.46226722]
  [0.40472    0.422333   0.6430686 ]
  [0.40547436 0.40914807 0.7081136 ]
  [0.43144175 0.43202645 0.89725167]
  [0.4330451  0.39967406 0.8786885 ]
  [0.43209535 0.45812854 0.78516227]
  [0.4342861  0.37604323 0.5373708 ]
  [0.43233085 0.48623252 0.723229  ]
  [0.4316029  0.35362387 0.7649404 ]
  [0.5143473  0.42545953 0.8942263 ]
  [0.51202106 0.4063034  0.85663503]
  [0.5798098  0.43092948 0.79041594]
  [0.5752194  0.40152025 0.91446793]
  [0.6350284  0.43530628 0.8818897 ]
  [0.62959176 0.39773783 0.8241108 ]]

 [[0.4037218  0.5827898  0.67246175]
  [0.39955136 0.5858912  0.5758649 ]
  [0.39989755 0.5801762  0.61855155]
  [0.40571553 0.5895947  0.6407122 ]
  [0.4064361  0.5764904  0.6190613 ]
  [0.43307802 0.6000275  0.85049653]
  [0.43165717 0.5661543  0.8947706 ]
  [0.43205553 0.6276468  0.7358969 ]
  [0.4337201  0.54073894 0.7524129 ]
  [0.43320304 0.64848286 0.71489465]

# TensorFlow Lite

In [6]:
### Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

### Load Model
interpreter  = tf.lite.Interpreter(model_path = '../../model/movenet/lite-model_movenet_multipose_lightning_tflite_float16_1.tflite')
interpreter.allocate_tensors() #have to do when working on tflite model 

### Load Video
video_path = '../../data/videos/'
sampleVideo = video_path + 'sampleVideo.mp4'
basicFour = video_path + 'basic_four.mp4'
basicTwo = video_path + 'basic_two.mp4'
swingDiff = video_path + 'swing_diff.mp4'
swingSame = video_path + 'swing_same.mp4'
swingRaw = video_path + 'swingRaw.mp4'
temporalDifficult = video_path + 'temporalDifficultRaw.mp4'
temporalEasy = video_path + 'temporalEasyRaw.mp4'

### Draw EDGES
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

### Vector List
vectorList = [
    [0,1],
    [0,2],
    [1,3],
    [2,4],
    [3,5],
    [0,6],
    [1,7],
    [6,7],
    [6,8],
    [7,9],
    [8,10],
    [9,10]
]

# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)


def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 3, (0,255,0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


###
# Variables for drawing plot in real-time


if __name__ == "__main__":
    ### Variables
    numberOfPeople = 6
#     lamdaVal = 0.885

#     minBPD = 10.0
#     maxBPD = 0.0
    
#     saveNum = 0
    saveBPD = []

    ### Loading Video File
    cap = cv2.VideoCapture(temporalEasy)
    while cap.isOpened():
        ret, frame = cap.read()

        ### Variables for each frame
        BPD = []
        
        # Resize image
        img = frame.copy()
        img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
        input_img = tf.cast(img, dtype=tf.int32)
        
        # Detection section
        results = movenet(input_img)
        keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
        keypoints_with_scores = keypoints_with_scores[:numberOfPeople]
        keypoints_only = np.delete(keypoints_with_scores,2,2)
        keypoints_only_body = np.delete(keypoints_only, [0,1,2,3,4], 1)



        # Render keypoints 
        loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
        # loop_through_people(frame, [keypoints_with_scores[0]], EDGES, 0.1)    # Check for first person.....

        
        cv2.imshow('Movenet Multipose', frame)

        
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()